# <b><font color='lightgreen'> Python to VerilogA </font></b>

Having the model trained in the python framework is possible to acesse the weights and bias and other parameters of the network. This script provides functions capable to, given the input parameters, generate a verilogA code.

Work by: André Amaral & Nuno Lourenço (ICG-IT.Lx)

In [3]:
# Needed Libraries
import csv
import numpy as np

### <b><font color='blue'> Fully Connected Layers </font></b>


<b> How to get the weights and bias </b>

l = model.fc1.state_dict()

W = l['weight'].detach().cpu().numpy()

B = l['bias'].detach().cpu().numpy()


In [3]:
def FC_HiddenLayer_to_VerilogA(W, B, input_dim, output_dim):
    """
    Implements the code for a Fully Connected Layer in Verilog-A
    
    :param W: Weight matrix for a specific layer.
    :type W: float

    :param B: Bias matrix for a specific layer.
    :type B: float

    :param input_dim: Number of neurons in the input layer.
    :type input_dim: float

    :param output_dim: Number of neurons in the output layer.
    :type output_dim: float
    """

    # ---------------------------- Get the Weights Vector in VerilogA Structure ----------------------------

    Pesos_VA = []

    for i in range(W.shape[1]):
        Wa = W[:, i]

        middle = []
        first = '{%e' % Wa[0]

        if (W.shape[0] != 1):
            middle_out = ',%e' % Wa[1]
            middle = first + middle_out

        last = '}'

        if (W.shape[0] > 2):
            for i in (n+2 for n in range(np.size(W, 0)-2)):
                aux = ',%e' % Wa[i]
                middle = middle + aux
            Pesos_VA.append(middle + last)
        elif (W.shape[0] == 2):
            Pesos_VA.append(middle + last)
        elif (W.shape[0] == 1):
            Pesos_VA.append(first + last)

    # ------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------
    # ---------------------------- Get the Bias Vector in VerilogA Structure -------------------------------
    middle = []
    first = '{%e' %B[0]

    if(B.shape[0] != 1):
        middle_out = ',%e' %B[1]
        middle = first + middle_out

    last = '}'

    if (B.shape[0] > 2):
        for i in (n+2 for n in range(np.size(B,0)-2)):
            aux = ',%e' %B[i]
            middle = middle + aux
        b1_VA = middle + last
    elif (B.shape[0] == 2):
        b1_VA = middle + last
    elif (B.shape[0] == 1):
        b1_VA = first + last

    # ------------------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------------------
    # ------------------------------------ Write the Script of VerilogA ------------------------------------
    neurons_number_next = np.size(W,0)-1
    neurons_number_prev = np.size(W,1)-1

    with open('FC_Layer.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module ANN_Middle_Layer(x,y);'])
        writer.writerow([''])
        writer.writerow(['  parameter integer x_dim = %i;' %input_dim])
        writer.writerow(['  parameter integer y_dim = %i;' %output_dim])
        
        for i in range(neurons_number_prev+1):
            writer.writerow(['  parameter real W2%i[0:%i] = %s;' %(i,neurons_number_next,Pesos_VA[i])])
        
        writer.writerow(['  parameter real b[0:%i] = %s;' %(neurons_number_next,b1_VA)])                
        writer.writerow(['  real a2[0:%i];' %neurons_number_next])
        writer.writerow([''])
        writer.writerow(['  input [0:%i]x;' %neurons_number_prev])
        writer.writerow(['  voltage [0:%i]x;' %neurons_number_prev])
        writer.writerow(['  output [0:%i]y;' %neurons_number_next])
        writer.writerow(['  voltage [0:%i]y;' %neurons_number_next])
        writer.writerow(['  genvar i;'])
        writer.writerow([''])
        writer.writerow(['  analog begin'])

        #Initialize the output variable (since is use an acumulator)
        for i in range(neurons_number_next+1):
            writer.writerow(['      a2[%i] = 0;' %i])
        
        writer.writerow([''])

        #Write the forward path
        for i in range(neurons_number_next+1):
            for j in range(neurons_number_prev+1):
                writer.writerow(['      a2[%i] = V(x[%i]) * W2%i[%i] + a2[%i];' %(i,j,j,i,i)])
        
        writer.writerow([''])

        #Update the output voltage pin
        writer.writerow(['      for(i = 0; i<= %i; i = i + 1) begin' %neurons_number_next])
        writer.writerow(['          V(y[i]) <+ a2[i] + b[i];'])

        writer.writerow(['      end'])
        writer.writerow(['  end'])
        writer.writerow(['endmodule'])

### <b><font color='blue'> LSTM Layers </font> <font color='red'>(Not Fully Implemented)</font></b>


To get the internal weights and bias use:

<b>Weights Input:</b> <i>w_ii, w_if, w_ic, w_io = <model_name>.<lstm_layer_name>.weight_ih_l0.chunk(4, 0)</i>

<b>Weights Hidden:</b> <i>w_ii, w_if, w_ic, w_io = <model_name>.<lstm_layer_name>.weight_hh_l0.chunk(4, 0)</i>

In case of use DataParallel (train the model with multiple GPU's):

<b>Weights Input:</b> <i>w_ii, w_if, w_ic, w_io = <model_name>.<module><lstm_layer_name>.weight_ih_l0.chunk(4, 0)</i>

In [1]:
def LSTM_Layer_to_VerilogA():
    with open('LSTM_Layer.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module LSTM_Layer(H_prev, C_prev, x, H_out, C_out);'])
        writer.writerow(['  	input H_prev, C_prev, x;'])
        writer.writerow(['  	voltage H_prev, C_prev, x;'])
        writer.writerow(['  	output H_out, C_out;'])
        writer.writerow(['  	voltage H_out, C_out;'])
        writer.writerow([''])
        writer.writerow(['  	real C_til;'])
        writer.writerow(['  	real O_t_aux;'])
        writer.writerow(['  	real O_t;'])
        writer.writerow(['  	real I_t_aux;'])
        writer.writerow(['  	real I_t;'])
        writer.writerow(['  	real F_t_aux;'])
        writer.writerow(['  	real F_t;'])
        writer.writerow(['  	real C_t;'])
        writer.writerow(['  	real output_value;'])
        writer.writerow([''])
        writer.writerow(['  	parameter real Wf = ????;'])
        writer.writerow(['  	parameter real Wi = ????;'])
        writer.writerow(['  	parameter real Wo = ????;'])
        writer.writerow(['  	parameter real Wc = ????;'])
        writer.writerow(['  	parameter real Uf = ????;'])
        writer.writerow(['  	parameter real Ui = ????;'])
        writer.writerow(['  	parameter real Uo = ????;'])
        writer.writerow(['  	parameter real Uc = ????;'])
        writer.writerow(['  	parameter real bf = ????;'])
        writer.writerow(['  	parameter real bi = ????;'])
        writer.writerow(['  	parameter real bo = ????;'])
        writer.writerow(['  	parameter real bc = ????;'])
        writer.writerow([''])
        writer.writerow(['  	parameter real HiddenInit = ????;'])
        writer.writerow(['  	parameter real CellInit = ????;'])
        writer.writerow([''])
        writer.writerow(['  	analog begin'])
        writer.writerow([''])
        writer.writerow(['  	    V(H_prev) <+ Hidden;'])
        writer.writerow(['  	    V(H_prev) <+ Cell;'])
        writer.writerow([''])
        writer.writerow(['  	    C_til = tanh(Wc * V(x) + Uc * V(H_prev) + bc);'])
        writer.writerow([''])
        writer.writerow(['  	    O_t_aux = Wo * V(x) + Uo * V(H_prev) + bo;'])
        writer.writerow(['  	    O_t = 1/(1+exp(-O_t_aux));'])
        writer.writerow([''])
        writer.writerow(['  	    I_t_aux = Wi * V(x) + Ui * V(H_prev) + bi;'])
        writer.writerow(['  	    I_t = 1/(1+exp(-I_t_aux));'])
        writer.writerow([''])
        writer.writerow(['  	    F_t_aux = Wf * V(x) + Uf * V(H_prev) + bf;'])
        writer.writerow(['  	    F_t = 1/(1+exp(-F_t_aux));'])
        writer.writerow([''])
        writer.writerow(['  	    C_t = F_t * V(C_prev) + I_t * C_til;'])
        writer.writerow([''])
        writer.writerow(['  	    output_value = O_t * tanh(V(C_out));'])
        writer.writerow([''])
        writer.writerow(['  	end'])
        writer.writerow([''])
        writer.writerow(['  	V(C_out) <+ C_t;'])
        writer.writerow(['  	V(C_prev) <+ C_t;'])
        writer.writerow(['  	V(H_out) <+ output_value;'])
        writer.writerow(['  	V(H_prev) <+ output_value;'])
        writer.writerow([''])
        writer.writerow(['endmodule'])


### <b><font color='blue'> Activation Functions </font></b>


#### <b><font color=#27BFC1> ReLU </font></b>

In [ ]:
 # ------------------------------------ Write the Script of VerilogA ------------------------------------
def ReLU_to_VerilogA():
    with open('ReLU.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module ReLU(n_deact, n_act);'])
        writer.writerow(['  input n_deact;'])
        writer.writerow(['  output n_act;'])
        writer.writerow(['  eletrical n_deact, n_act;'])
        writer.writerow(['  real result;'])
        writer.writerow(['  real p_num;'])
        writer.writerow([' '])
        writer.writerow(['  analog begin'])
        writer.writerow(['      p_num = V(n_deact);'])
        writer.writerow(['      if(p_num >= 0)'])
        writer.writerow(['          result = p_num;'])
        writer.writerow(['      else'])
        writer.writerow(['          result = 0.0;'])
        writer.writerow(['      V(n_act) <+ result;'])
        writer.writerow(['  end'])
        writer.writerow(['endmodule;'])

#### <b><font color=#27BFC1> ELU </font></b>

In [ ]:
 # ------------------------------------ Write the Script of VerilogA ------------------------------------
def ELU_to_VerilogA():
    with open('ELU.VA', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['`include "constants.vams"'])
        writer.writerow(['`include "disciplines.vams"'])
        writer.writerow([''])
        writer.writerow(['module ELU(n_deact, n_act);'])
        writer.writerow(['  input n_deact;'])
        writer.writerow(['  output n_act;'])
        writer.writerow(['  eletrical n_deact, n_act;'])
        writer.writerow(['  real result;'])
        writer.writerow(['  real p_num;'])
        writer.writerow(['  parameter real alpha = 1;'])
        writer.writerow([' '])
        writer.writerow(['  analog begin'])
        writer.writerow(['      p_num = V(n_deact);'])
        writer.writerow(['      if(p_num > 0)'])
        writer.writerow(['          result = p_num;'])
        writer.writerow(['      else'])
        writer.writerow(['          result = alpha * (exp(p_num) - 1);'])
        writer.writerow(['      V(n_act) <+ result;'])
        writer.writerow(['  end'])
        writer.writerow(['endmodule;'])

### <b><font color='blue'> Extra Modules </font></b>


#### <b><font color=#27BFC1> Dimensions Concatenator </font></b>

In [ ]:
def Concat(dims_vector, num_dims, neurons_number_prev):
    """
    Concatenate the dimensions of the circuits to the output of the previous layer.
    
    :param dims_vector: Vector with the circuit dimensions
    :type dims_vector: float

    :param num_dims: Length of the dims_vector
    :type num_dims: int

    :param num_neurons_prev: Number of neurons in the previous layer.
    :type num_dims: int
    """
    
    # ---------------------------- Get the Dimension Vector in VerilogA Structure ----------------------------
    Pesos_VA = []
    Wa = dims_vector
    Wa = dims_vector[0,:]

    middle = []
    first = '{%e' % Wa[0]

    middle_out = ',%e' % Wa[1]
    middle = first + middle_out

    last = '}'

    for i in (n+2 for n in range(np.size(Wa, 0)-2)):
        aux = ',%e' % Wa[i]
        middle = middle + aux
    Pesos_VA.append(middle + last)
    
     # ------------------------------------ Write the Script of VerilogA ------------------------------------
    with open('Concat.va', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['// z1: Is the output from the previous FC Layer'])
        writer.writerow(['// z1_dims: Is the concatenation result'])
        writer.writerow([''])
        writer.writerow(['module Concatenator(z1, z1_dims);'])
        writer.writerow([''])
        writer.writerow(['  input [0:%i]z1;' %(neurons_number_prev-1)])
        writer.writerow(['  voltage [0:%i]z1;' %(neurons_number_prev-1)])
        writer.writerow(['  output [0:%i]z1_dims;' %((neurons_number_prev+num_dims)-1)])
        writer.writerow(['  voltage [0:%i]z1_dims;' %((neurons_number_prev+num_dims)-1)])
        writer.writerow(['  parameter real Wd[0:%i] = %s;' %((num_dims-1), Pesos_VA[0])])
        writer.writerow(['  real b[0:%i];' %((neurons_number_prev+num_dims)-1)])
        writer.writerow(['  genvar i;'])
        writer.writerow([''])
        writer.writerow(['  analog begin'])
        writer.writerow(['      for(i = 0; i<= %i; i = i + 1) begin' %(neurons_number_prev-1)])
        writer.writerow(['          b[i] = V(z1[i]);'])
        writer.writerow(['          V(z1_dims[i]) <+ b[i];'])
        writer.writerow(['      end'])
        writer.writerow([''])
        writer.writerow(['      for(i = 0; i<= %i; i = i + 1) begin' %(num_dims-1)])
        writer.writerow(['          b[%i + i] = Wd[i];' %(neurons_number_prev)])
        writer.writerow(['          V(z1_dims[%i + i]) <+ b[%i + i];' %(neurons_number_prev, neurons_number_prev)])
        writer.writerow(['      end'])
        writer.writerow(['  end'])
        writer.writerow(['endmodule'])

#### <b><font color=#27BFC1> Delay Lines </font></b>

In [38]:

def DLines(num_delays, time_delay):
    """
    Apply a delay line to the input waves.
    
    :param num_delays: Vector with the circuit dimensions
    :type num_delays: int

    :param time_delay: Value of the delay between samples
    :type time_delay: float
    """
    # ------------------------------------ Write the Script of VerilogA ------------------------------------
    with open('DLines.va', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['module DelayLines(in_wave, out_wave);'])
        writer.writerow([''])
        writer.writerow(['  input in_wave;'])
        writer.writerow(['  voltage in_wave;'])
        writer.writerow(['  output [0:%i]out_wave;' %(num_delays)])
        writer.writerow(['  voltage [0:%i]out_wave;' %(num_delays)])
        writer.writerow(['  real delay_wave;'])
        writer.writerow(['  real a;'])
        writer.writerow(['  real time_delay;'])
        writer.writerow(['  real b[0:%i];' %(num_delays-1)])
        writer.writerow(['  parameter real fixed_delay= %f;' %time_delay])
        writer.writerow([''])
        writer.writerow(['  analog begin'])
        writer.writerow(['      time_delay = fixed_delay;'])
        writer.writerow(['      a = V(in_wave);'])
        writer.writerow([''])
        for i in range(num_delays):
            writer.writerow(['      b[%i] = absdelay(a, (%i)*time_delay);' %(i,(i+1))])
        writer.writerow([''])
        writer.writerow(['      V(out_wave[0]) <+ a;'])
        for i in range (0, num_delays):
            writer.writerow(['      V(out_wave[%i]) <+ b[%i];' %((i+1),i)])
        writer.writerow([''])
        writer.writerow(['  end'])
        writer.writerow(['endmodule'])
    

#### <b><font color=#27BFC1> Normalizer (MinMaxScaler) </font></b>

In [46]:
def NormMMS(feature_desired_range_max, features_desired_range_min, min, max):
    """
    Apply the MinMaxScaler normalizer to the input waves
    
    :param feature_range_max: Maximum desired value of the sacled feature
    :type feature_range_max: int

    :param feature_range_min: Minumum desired value of the sacled feature
    :type feature_range_min: int

    """
    # ------------------------------------ Write the Script of VerilogA ------------------------------------
    with open('NormMMS.va', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['module NormMMS(in_wave, out_wave);'])
        writer.writerow([''])
        writer.writerow(['  input in_wave;'])
        writer.writerow(['  voltage in_wave;'])
        writer.writerow(['  output out_wave;'])
        writer.writerow(['  voltage out_wave;'])
        writer.writerow([''])
        writer.writerow(['  real parameter aux_for_min = %f;' %min])
        writer.writerow(['  real parameter aux_for_max = %f;' %max])
        writer.writerow(['  real x_std;'])
        writer.writerow(['  real x_scaled;'])
        writer.writerow([''])
        writer.writerow(['  analog begin'])
        writer.writerow([''])
        # Obtain the standard deviation
        writer.writerow(['      x_std = (V(in_wave)-aux_for_min_set) / (aux_for_max_set - aux_for_min_set);'])
        writer.writerow(['      x_scaled = x_std * (%i - %i) + %i;' %(feature_desired_range_max, features_desired_range_min, features_desired_range_min)])
        writer.writerow([''])
        # Assign the x_sacled to the output variable
        writer.writerow(['      V(out_wave) <+ x_scaled;']) 
        writer.writerow([''])
        writer.writerow(['  end'])
        writer.writerow(['endmodule'])


#### <b><font color=#27BFC1> De-Normalizer (MinMaxScaler) </font></b>

In [4]:
def DeNormMMS(feature_desired_range_max, feature_desired_range_min, min, max):
    """
    Apply the MinMaxScaler de-normalizer to the input waves
    
    :param feature_range_max: Maximum desired value of the sacled feature
    :type feature_range_max: int

    :param feature_range_min: Minumum desired value of the sacled feature
    :type feature_range_min: int

    """
    # ------------------------------------ Write the Script of VerilogA ------------------------------------
    with open('DeNormMMS.va', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['module NormMMS(in_wave, out_wave);'])
        writer.writerow([''])
        writer.writerow(['  input in_wave;'])
        writer.writerow(['  voltage in_wave;'])
        writer.writerow(['  output out_wave;'])
        writer.writerow(['  voltage out_wave;'])
        writer.writerow([''])
        writer.writerow(['  real parameter aux_for_min = %f;' %min])
        writer.writerow(['  real parameter aux_for_max = %f;' %max])
        writer.writerow(['  real X;'])
        writer.writerow([''])
        writer.writerow(['  analog begin'])
        writer.writerow([''])
        # Obtain the standard deviation
        writer.writerow(['      X = ((V(in_wave) * (%f - %f))/((%i - %i) + %i)) + %f;' %(max, min, feature_desired_range_max, feature_desired_range_min, feature_desired_range_min, min )])
        writer.writerow([''])
        # Assign the x_sacled to the output variable
        writer.writerow(['      V(out_wave) <+ X;']) 
        writer.writerow([''])
        writer.writerow(['  end'])
        writer.writerow(['endmodule'])

#### <b><font color=#27BFC1> Gaussian Noise </font></b>

In [7]:
def GaussianNoise(mean, stdv, seed):
    """
    Apply the gaussian noise with specified mean and standard deviation to the input wave
    
    :param feature_range_max: Maximum desired value of the sacled feature
    :type feature_range_max: int

    :param feature_range_min: Minumum desired value of the sacled feature
    :type feature_range_min: int

    """
    # ------------------------------------ Write the Script of VerilogA ------------------------------------
    with open('GaussNoise.va', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['module GaussianNoise(in_wave, out_wave);'])
        writer.writerow([''])
        writer.writerow(['  input in_wave;'])
        writer.writerow(['  voltage in_wave;'])
        writer.writerow(['  output out_wave;'])
        writer.writerow(['  voltage out_wave;'])
        writer.writerow([''])
        writer.writerow(['  parameter real mean = %f;' %mean])
        writer.writerow(['  parameter real stdv = %f;' %stdv])
        writer.writerow(['  parameter integer seed = %i;' %seed])
        writer.writerow(['  real a;'])
        writer.writerow(['  real b;'])
        writer.writerow([''])
        writer.writerow(['  analog begin'])
        writer.writerow([''])
        writer.writerow(['      a = $rdist_normal(seed, mean, stdv);'])
        writer.writerow(['      b = V(in_wave) - a;'])
        writer.writerow(['      V(out_wave) <+ b;']) 
        writer.writerow([''])
        writer.writerow(['  end'])
        writer.writerow(['endmodule'])